# To be run once to setup modules and paths


In [ ]:
import os
import numpy as np
import fiona
from shapely.geometry import shape

import subprocess
import shlex



In [ ]:
sorted_products = products.sort_values(["relativeorbitnumber", "orbitdirection", "slicenumber", "ingestiondate"], ascending=[True, True, True, True]) #df1.sort(['a', 'b'], ascending=[True, False])
pairs = {}
pair_list = []
downloadfolder = os.path.join(basefolder, "s1slc")

import requests
import fcntl
from IPython.display import clear_output
import time

def download_file(download_url, filename):
    args = [
            "wget",
            '%s'%(download_url),
            "--user",
            "helpdesk@pixalytics.com",
            "--password",
            "Pixalytics123!",
            "-O",
            '%s.zip'%(filename),
            ]

    # create the subprocess
    p = subprocess.Popen(args,
                         stdout=subprocess.PIPE,
                         stderr=subprocess.STDOUT,
                         bufsize=1,
                         universal_newlines=True)

    # forward messages from stdout and stderr onto the console
    with p.stdout as stdout:
        for line in iter(stdout.readline, b""):
            if line == "":
                break
            clear_output(wait=True)
            print("Downloading %s"%(filename), line.rstrip())

    # wait to exit and retreieve the exit code
    exit_code = p.wait()

    # raise an exception if 'gpt' return an unexpected exit code
    if exit_code != 0:
        raise RuntimeError("Non-zero return code from GPT.")

def batch_download_from_archive(filtered_products, downloadfolder):
    currently_requested = None
    for download_num, (id1, data) in enumerate(filtered_products.iterrows()):
        session = requests.Session()
        session.auth = ('helpdesk@pixalytics.com', 'Pixalytics123!')
        #print(id, data['title'])
        print(download_num)
        waiting_for = 0
        while True:
            queryurl = """https://catalogue.onda-dias.eu/dias-catalogue/Products(%s)"""%(id1)
            offline = session.get(queryurl)
            offline = offline.text.split("offline")[1][2:6]
            if offline == "true":
                offline = True 
            elif offline == "fals":
                offline = False
            if offline:
                clear_output(wait=True)
                print("Downloaded %s of %s files"%(download_num, len(filtered_data)))
                print("Requested %s, id = %s"%(data['title'], id1))
                print("Waiting for %s minutes"%(waiting_for))
                if id1 == currently_requested:
                    time.sleep(300)
                    waiting_for += 5
                else:
                    requested = session.post("https://catalogue.onda-dias.eu/dias-catalogue/Products(%s)/Ens.Order"%(id1))
                    currently_requested = id1
            else:
                download_url = """https://catalogue.onda-dias.eu/dias-catalogue/Products(%s)/$value"""%(id1)
                download_file(download_url, os.path.join(downloadfolder, data['title']))
                clear_output(wait=True)
                print("Downloaded %s of %s files"%(download_num, len(filtered_data)))
                print("Requested %s, id = %s"%(data['title'], id1))
                break
            """
            if requested.status_code == 429: #too many requests error
                clear_output(wait=True)
                print("Too many requests in the last hour, will try again soon for %s"%(id))
                time.sleep(300)
                continue
            requested_response = requested.json()
            if (requested.status_code == 403) & (requested_response['error']['message'] == 'Could not order an online product'): # This error says {"error":{"code":"001","message":"Could not order an online product"}} - but lets check
                download_url = "https://catalogue.onda-dias.eu/dias-catalogue/Products(%s)/$value"%(id1)
                download_file(download_url, os.path.join(downloadfolder, data['title']))
                break
            elif requested.status_code == 401: # if the session has ended - restart it 
                session = requests.Session()
                session.auth = ('helpdesk@pixalytics.com', 'Pixalytics123!')
            else:
                clear_output(wait=True)
                print("Downloaded %s of %s files"%(download_num, len(products)))
                print("Requested %s, id = %s"%(data['title'], id1))
                print(requested, requested_response)
            time.sleep(60)
            """
for n, (i, p) in enumerate(sorted_products.iterrows()):
    current_slice = p['slicenumber']
    if n == 0:
        last_slice = current_slice
    if current_slice != last_slice:  
        pairs[last_orbit, last_slice] = pair_list
        pair_list = []
    pair_list.append(p['title'])
    last_slice = p['slicenumber']
    last_orbit = p['relativeorbitnumber']
    #print(p['polarisationmode'], p['relativeorbitnumber'], p['orbitdirection'], p['ingestiondate'], p['slicenumber'])

# first set it up to download files
#for pairings, title_list in pairs.items():
#pairing_data = products[products['title'].isin(title_list)]
filtered_data = check_for_existing_downloads(products, downloadfolder) 
batch_download_from_archive(filtered_data, downloadfolder)
"""
    ### download_files
    products_to_download = filtered_data.index.to_list()
    # unhash the below lines when the download folder is properly set!
    if pairings != (23, 5):
        continue
    for p in products_to_download:
        #api.download(p, directory_path=downloadfolder)
            continue
    
print(os.listdir(basefolder))
#for d, p in products.iterrows():
#    print(p['title'], p['relativeorbitnumber'], p['orbitdirection'], p['slicenumber'])
"""

In [ ]:
#print(os.listdir(os.path.join(basefolder, "s1slc")))
#os.remove(os.path.join(basefolder, "s1slc/S1B_IW_SLC__1SDV_20190201T064521_20190201T064548_014749_01B806_77D9.zip"))

In [ ]:
gpt = '/opt/snap/bin/gpt'
# Choose the pairing and prepare the properties files...
#file1 = os.path.join(os.getcwd(), 'temp_data/S1A_IW_SLC__1SDV_20201011T064608_20201011T064635_034745_040C5C_E064.zip')
file1 = os.path.join(os.getcwd(), 'temp_data/S1A_IW_SLC__1SDV_20201023T064607_20201023T064634_034920_04126F_BF03.zip')
file2 = os.path.join(os.getcwd(), 'temp_data/S1A_IW_SLC__1SDV_20201104T064607_20201104T064634_035095_041874_B5FE.zip')
xml_folder = os.path.join(os.getcwd(), "SLC_Data_processing/XML_files/")
intermediate_path = os.path.join(os.getcwd(), "SLC_Data_processing/Intermediate_files/")
processed_folder =  os.path.join(os.getcwd(), "SLC_Data_processing/Processed_images/")

outputname = os.path.basename(file1).split("_")[5] + "_" + os.path.basename(file2).split("_")[5]
outputname = os.path.join(processed_folder, outputname+".tif")                  
assert os.path.isdir(processed_folder), "Unable to locate '{}'.".format(processed_folder) 

if os.path.isfile(outputname):
    processed_file_exists = True
else:
    processed_file_exists = False
processed_file_exists = False
# lets see if we have already processed this combination
if not processed_file_exists:
    # Lets do it for each swath
    input1 = "input1="+file1
    input2 = "input2="+file2
    outputs = []
    for swathnum in range(1, 4):
        output = "output=" + intermediate_path + "IW%s"%(swathnum)
        swath="swath=IW%s"%(swathnum)
        f = open(os.path.join(xml_folder, "subswath%s.properties"%(swathnum)), "w")
        f.write(input1+"\n")
        f.write(input2+"\n")
        f.write(output+"\n")
        f.write(swath+"\n")
        f.close()
        outputs.append("input%s="%(swathnum) + intermediate_path + "IW%s.dim"%(swathnum))
    # finally lets write the Mergeswaths.properties file
    f = open(os.path.join(xml_folder, "Mergeswaths.properties"), "w")
    f.write(outputs[0]+"\n")
    f.write(outputs[1]+"\n")
    f.write(outputs[2]+"\n")
    f.write("output=" + outputname.strip(".tif"))
    f.close()

In [ ]:
# Process the individual subswaths
if not processed_file_exists:
    xml_filename = os.path.join(xml_folder, 'ProcessSubswath.xml')
    assert os.path.exists(gpt), "Unable to find 'gpt' executable."
    assert os.path.isfile(xml_filename), "Unable to locate '{}'.".format(xml_filename)

    #Lets loop through and process the individual swaths
    properties_files = [os.path.join(xml_folder, file) for file in os.listdir(xml_folder) if (file.endswith(".properties")) and (file.startswith("subswath"))]
    # create the subprocess
    for propfile in properties_files:
        print(propfile)
        args = [
                gpt,
                xml_filename,
                "-p",
                propfile
            ]

        # create the subprocess
        p = subprocess.Popen(args,
                             stdout=subprocess.PIPE,
                             stderr=subprocess.STDOUT,
                             bufsize=1,
                             universal_newlines=True)

        # forward messages from stdout and stderr onto the console
        with p.stdout as stdout:
            for line in iter(stdout.readline, b""):
                if line == "":
                    break
                print(line.rstrip())

        # wait to exit and retreieve the exit code
        exit_code = p.wait()

        # raise an exception if 'gpt' return an unexpected exit code
        if exit_code != 0:
            raise RuntimeError("Non-zero return code from GPT.")

In [ ]:
# Merge the individual subswaths
if not processed_file_exists:
    print("Merging subswaths and saving to %s"%(outputname))
    xml_filename = os.path.join(xml_folder, 'Mergeswaths.xml')
    propfile = os.path.join(xml_folder, 'Mergeswaths.properties')
    args = [
            gpt,
            xml_filename,
            "-p",
            propfile
        ]

    # create the subprocess
    p = subprocess.Popen(args,
                         stdout=subprocess.PIPE,
                         stderr=subprocess.STDOUT,
                         bufsize=1,
                         universal_newlines=True)

    # forward messages from stdout and stderr onto the console
    with p.stdout as stdout:
        for line in iter(stdout.readline, b""):
            if line == "":
                break
            print(line.rstrip())

    # wait to exit and retreieve the exit code
    exit_code = p.wait()

    # raise an exception if 'gpt' return an unexpected exit code
    if exit_code != 0:
        raise RuntimeError("Non-zero return code from GPT.")

In [ ]:
print(os.listdir(datasets))